# Data Anaylsis:

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("TicketCounts").getOrCreate()
# Read the input data from CSV file
df = spark.read.csv("D:/Users/Rahul/Parking_Violations_Issued_-_Fiscal_Year_2023.csv", header=True)
# Print the schema of the DataFrame
df.printSchema()
#"C:/Users/Lenovo/Desktop/shot_logs.csv"

root
 |-- Summons Number: string (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Plate Type: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Violation Code: string (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Issuing Agency: string (nullable = true)
 |-- Street Code1: string (nullable = true)
 |-- Street Code2: string (nullable = true)
 |-- Street Code3: string (nullable = true)
 |-- Vehicle Expiration Date: string (nullable = true)
 |-- Violation Location: string (nullable = true)
 |-- Violation Precinct: string (nullable = true)
 |-- Issuer Precinct: string (nullable = true)
 |-- Issuer Code: string (nullable = true)
 |-- Issuer Command: string (nullable = true)
 |-- Issuer Squad: string (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- Time First Observed: string (nullable = true)
 |-- Violation County: str

# Problem1
When are tickets most likely to be issued? (15 pts)

In [22]:
# By thw analysis of Violation Time
from pyspark.sql.functions import count, col

# Read the CSV file into a DataFrame
df = spark.read.csv("D:/Users/Rahul/Parking_Violations_Issued_-_Fiscal_Year_2023.csv", header=True)

# group by violation time and count number of violations
grouped_df = df.groupBy("Violation Time").agg(count("*").alias("NumViolations"))

# sort the result in descending order of the number of violations
result_df = grouped_df.orderBy(col("NumViolations").desc())

# display the result
result_df.show()


+--------------+-------------+
|Violation Time|NumViolations|
+--------------+-------------+
|         0836A|        22958|
|         0839A|        21772|
|         0906A|        21712|
|         0840A|        21588|
|         0838A|        21585|
|         0837A|        21074|
|         0841A|        20883|
|         0908A|        20765|
|         0842A|        20660|
|         1141A|        20564|
|         1140A|        20502|
|         1142A|        20497|
|         0909A|        20488|
|         0843A|        20473|
|         0910A|        20421|
|         0907A|        20249|
|         1138A|        20171|
|         1139A|        20156|
|         0844A|        20152|
|         0845A|        20089|
+--------------+-------------+
only showing top 20 rows



Based on the data given, we can conclude that tickets are most likely to be issued at 8:36 AM (0836A)

In [26]:
# By the anaylsis of Issue Date 
from pyspark.sql.functions import count
from pyspark.sql.functions import to_date

# Read the CSV file into a DataFrame
df = spark.read.csv("D:/Users/Rahul/Parking_Violations_Issued_-_Fiscal_Year_2023.csv", header=True)

# Convert the "Issue Date" column to a date type
df = df.withColumn("Issue Date", to_date("Issue Date", "MM/dd/yyyy"))

# Group the data by "Issue Date" and count the number of tickets issued on each date
df_grouped = df.groupBy("Issue Date").agg(count("*").alias("Num Tickets Issued"))

# Sort the results in descending order by the number of tickets issued
df_sorted = df_grouped.orderBy("Num Tickets Issued", ascending=False)

# Show all rows of the sorted DataFrame
df_sorted.show(10)


+----------+------------------+
|Issue Date|Num Tickets Issued|
+----------+------------------+
|2022-08-04|             66726|
|2022-08-05|             65393|
|2022-08-02|             64876|
|2022-06-30|             64846|
|2022-07-19|             64815|
|2022-11-25|             64411|
|2022-08-11|             64192|
|2022-08-18|             63975|
|2022-07-12|             63780|
|2022-07-15|             63646|
+----------+------------------+
only showing top 10 rows



we can conclude that August 4, 2022 (Aug Month), is the day when tickets are most likely to be issued

# Problem 2
What are the most common years and types of cars to be ticketed? (15 pts)

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AppName").getOrCreate()
from pyspark.sql.functions import desc, count

# Read the CSV file into a DataFrame
df = spark.read.csv("D:/Users/Rahul/Parking_Violations_Issued_-_Fiscal_Year_2023.csv", header=True)

# Group the data by "Vehicle Year" and "Vehicle Make", and count the number of tickets for each group
df_grouped = df.groupBy("Vehicle Year", "Vehicle Make").agg(count("*").alias("Num Tickets"))

# Sort the results in descending order by the number of tickets
df_sorted = df_grouped.sort(desc("Num Tickets"))

# Show the top 10 rows of the sorted DataFrame
df_sorted.show(10)

+------------+------------+-----------+
|Vehicle Year|Vehicle Make|Num Tickets|
+------------+------------+-----------+
|           0|       HONDA|     208547|
|           0|        FORD|     168583|
|           0|       TOYOT|     165496|
|           0|       NISSA|     139312|
|        2021|       TOYOT|     117999|
|        2019|       HONDA|     113890|
|        2021|       HONDA|     107202|
|        2020|       HONDA|     104349|
|           0|       CHEVR|      99590|
|           0|         BMW|      95905|
+------------+------------+-----------+
only showing top 10 rows



In [6]:
# Filter out rows where Vehicle Year is 0 as these are null data 
df_filtered = df_grouped.filter(df_grouped["Vehicle Year"] != "0")

# Sort the results in descending order by the number of tickets
df_sorted = df_filtered.sort(desc("Num Tickets"))

# Show the top 10 rows of the sorted DataFrame
df_sorted.show(10)


+------------+------------+-----------+
|Vehicle Year|Vehicle Make|Num Tickets|
+------------+------------+-----------+
|        2021|       TOYOT|     117999|
|        2019|       HONDA|     113890|
|        2021|       HONDA|     107202|
|        2020|       HONDA|     104349|
|        2022|       TOYOT|      95782|
|        2022|       HONDA|      93183|
|        2018|       TOYOT|      91268|
|        2018|       HONDA|      91213|
|        2017|       TOYOT|      88740|
|        2019|        FORD|      86105|
+------------+------------+-----------+
only showing top 10 rows



In [ ]:
#See Whole data in ans folder
#df_sorted.show(df_sorted.count())


By understanding data we can say that TOYOTA cars in year  2021 has most number of ticket,in 2nd place it is Honda cars in 2019.In 3rd place it is again Honda cars in year 2021 who got 107202 tickets

# Problem 3
Where are tickets most commonly issued? (15 pts)

In [30]:
from pyspark.sql.functions import count, desc

# Read the CSV file into a DataFrame
df = spark.read.csv("D:/Users/Rahul/Parking_Violations_Issued_-_Fiscal_Year_2023.csv", header=True)

# Group the data by Violation County and count the number of violations
ticket_counts = df.groupBy("Violation County").agg(count("*").alias("num_tickets"))

# Sort the results in descending order and show the top 10 counties
top_counties = ticket_counts.orderBy(desc("num_tickets")).limit(10)
top_counties.show()

+----------------+-----------+
|Violation County|num_tickets|
+----------------+-----------+
|              NY|    2450153|
|              QN|    1858441|
|              BK|    1732079|
|              BX|    1497854|
|               K|    1365103|
|               Q|    1215635|
|              MN|     603098|
|              ST|     328334|
|           Kings|     188817|
|               R|      99407|
+----------------+-----------+



we can see that tickets are most commonly issued in New York City. The most ticketed county is NY (which likely refers to the borough of Manhattan), with 2,450,153 tickets issued. The next most ticketed counties are QN (Queens) and BK (Brooklyn), with 1,858,441 issued.

In [29]:
from pyspark.sql.functions import count, desc

# Read the CSV file into a DataFrame
df = spark.read.csv("D:/Users/Rahul/Parking_Violations_Issued_-_Fiscal_Year_2023.csv", header=True)

# Group the data by precinct and count the number of violations
ticket_counts = df.groupBy("Violation Precinct").agg(count("*").alias("num_tickets"))

# Sort the results in descending order and show the top 10 precincts
top_precincts = ticket_counts.orderBy(desc("num_tickets")).limit(10)
top_precincts.show(top_precincts.count())

+------------------+-----------+
|Violation Precinct|num_tickets|
+------------------+-----------+
|                 0|    5349526|
|                19|     282466|
|                13|     254057|
|                 6|     224686|
|               114|     221523|
|                14|     190012|
|                18|     176733|
|                 9|     162228|
|                 1|     152429|
|               109|     137833|
+------------------+-----------+



By analysis of Violation Precinct,The precinct numbered 0 has the highest number of tickets issued, which is 5349526. This could indicate that either there is some error in recording the precinct number, or this number is being used as a default value for tickets with unknown precincts.In NY we can say that ,The precincts numbered 19 and 13 have the second and third highest number of tickets issued respectively, with 282466 and 254057 tickets each.

# Problem 4
Which color of the vehicle is most likely to get a ticket? (15 pts)

In [13]:
from pyspark.sql.functions import desc, count

# Read the CSV file into a DataFrame
df = spark.read.csv("D:/Users/Rahul/Parking_Violations_Issued_-_Fiscal_Year_2023.csv", header=True)

# Group the data by vehicle color and count the number of tickets
color_counts = df.groupBy("Vehicle Color").agg(count("*").alias("Num Tickets"))

# Sort the results in descending order and show the top color
top_color = color_counts.sort(desc("Num Tickets")).first()
print("The color most likely to get a ticket is: ", top_color["Vehicle Color"])


The color most likely to get a ticket is:  GY


In [11]:
from pyspark.sql.functions import col

# group by vehicle color and count the number of violations for each color
grouped_df = parking_df.groupBy("Vehicle Color") \
                      .agg({"Violation Code": "count"}) \
                      .withColumnRenamed("count(Violation Code)", "NumViolations")

# sort by the number of violations in descending order and display the first row
result_df = grouped_df.orderBy(col("NumViolations").desc())
result_df.show(1)


+-------------+-------------+
|Vehicle Color|NumViolations|
+-------------+-------------+
|           GY|      2275457|
+-------------+-------------+
only showing top 1 row



The color most likely to get a ticket is:  GY with 2275457 NumViolations

# Problem 5
Based on a K-Means algorithm, please try to answer the following question:
• Given a Black vehicle parking illegally at 34510, 10030, 34050 (street codes). What is
the probability that it will get an ticket? (very rough prediction). (20 pts)

In [12]:
df = spark.read.format("csv").option("header", "true").load("D:/Users/Rahul/Parking_Violations_Issued_-_Fiscal_Year_2023.csv")
black_vehicles = df.filter(df["Vehicle Color"] == "BLACK")
from pyspark.sql.functions import pow, sqrt
black_vehicles = black_vehicles.withColumn("Distance", sqrt(pow(black_vehicles["Street Code1"]-34510, 2) +\
                                                             pow(black_vehicles["Street Code2"]-10030, 2) + pow(black_vehicles["Street Code3"]-34050, 2)))
selected_columns = ["Distance", "Violation Code"]
black_vehicles = black_vehicles.select(selected_columns)

In [13]:
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType
black_vehicles = black_vehicles.withColumn("Violation Code", col("Violation Code").cast(DoubleType()))
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=selected_columns, outputCol="features")
kmeans_data = assembler.transform(black_vehicles).select("features")

In [14]:
from pyspark.ml.clustering import KMeans
kmeans = KMeans(k=3, seed=1)
model = kmeans.fit(kmeans_data)
from pyspark.ml.linalg import Vectors
new_point = Vectors.dense([0.1, 28])
predicted_cluster = model.predict(new_point)



In [15]:
all_clusters = model.transform(kmeans_data)
predicted_cluster_data = all_clusters.filter(all_clusters["prediction"] == predicted_cluster)
count_predicted_cluster = predicted_cluster_data.count()
total_count = all_clusters.count()
probability = count_predicted_cluster / total_count


In [16]:
print("The probability of getting a ticket for a black vehicle parking illegally at 34510, 10030, 34050 is", probability)

The probability of getting a ticket for a black vehicle parking illegally at 34510, 10030, 34050 is 0.4452690210726885
